### 풀이
- LCA의 부모를 구할 때 1개의 부모만을 기록했던 것을 2^i번째 부모를 기록하도록 변경하는 아이디어를 사용한다.


- 우선 주워진 입력으로부터(root 노드가 1) 각 노드의 level을 기록할 방법에 대해서 생각해보자.
  - 루트 노드에서 BFS로 탐색하면 각 노드의 level을 기록할 수 있다. (어찌보면 "거리"로 생각할 수도 있다)
  - 연결 정보의 쌍 `(a, b)`에서, 반드시 `a`가 `b`의 부모라는 표현이 없으므로 연결 정보를 입력할 때 `a`와 `b`를 서로 이어놓고, 순회할 때 `visited`도 써야한다. 아마도.
  - 트리 문제는 이렇게 전처리 과정이 들어간다고 한다.

In [ ]:
def dfs(T, level) :
  vis = [False] * (N+1)
  vis[1] = True
  Q = [1]
  l = 1
  while Q :
    nextQ = []
    while Q:
      u = Q.pop()
      for v in T[u] :
        if not vis[v] :
          vis[v] = True
          level[v] = l
          nextQ.append(v)
    Q = nextQ
    l += 1

- 희소 테이블을 만들어야 한다. 
  - 연결정보를 이용해 트리를 구성하면서 각 노드의 (1번째)부모 노드를 구한다.

In [ ]:
import sys, io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

N = int(input())
T = [[] for _ in range(N+1)]
P = [-1] * (N+1) #부모 노드 
L = [0] * (N+1) #노드의 높이
for _ in range(N-1) :
  a, b = map(int, input().split())
  T[a].append(b)
  T[b].append(a)

def dfs(T, P, L) : #각 노드에 대해 부모 노드와 노드의 높이를 저장
  vis = [False] * (N+1)
  vis[1] = True
  Q = [1]
  l = 1
  while Q :
    nextQ = []
    while Q:
      u = Q.pop()
      for v in T[u] :
        if not vis[v] :
          vis[v] = True
          P[v] = u
          L[v] = l
          nextQ.append(v)
    Q = nextQ
    l += 1

dfs(T, P, L)
MAX = 18 #2^18 > 100000
DP = [P] + [[0] * (N+1) for _ in range(MAX)] #DP[i][j] : j의 2^i번째 부모
for i, l in enumerate(DP[1:], 1) : #O(NlogN)
  for j in range(N+1) :
    l[j] = DP[i-1][DP[i-1][j]]

for _ in range(int(input())) :
  a, b = map(int, input().split())
  
  if L[a] < L[b] : #a가 더 깊은 노드가 되도록
    a, b = b, a
  
  for i in range(MAX-1, -1, -1) : #a와 b의 높이를 맞춰준다. (logN)
    if L[a] - L[b] >= (1 << i) :
      a = DP[i][a]

  #두 정점의 높이가 같으므로 2^i와 2^(i+1) 안에 공통 조상이 존재한다. 즉, P에서 찾을 수 있게된다.
  #2^i씩 동시에 올라가면서 공통 조상을 찾는다. (logN)
  if a != b:
    for i in range(MAX-1, -1, -1) :
      # print(i)
      if DP[i][a] != -1 and DP[i][a] != DP[i][b] :
        a = DP[i][a]
        b = DP[i][b]
    a = DP[0][a] #a와 b의 부모가 같으므로, a의 부모를 출력한다.
  sys.stdout.write(str(a) + '\n')

- a와 b의 높이를 맞추는 아이디어를 다시 잘 읽어볼 필요가 있다.